In [1]:
import numpy as np
import pandas as pd 
from gurobipy import *

In [29]:
train = pd.read_csv("/Users/estherchung/Downloads/AirbnbTrain.csv") 
test = pd.read_csv("/Users/estherchung/Downloads/AirbnbTest.csv") 

In [5]:
train.head()

,latitude,longitude,Entire home,accommodates,bathrooms,bedrooms,beds,cleaning_fee,minimum_nights,number_of_reviews,review_scores_rating,instant_bookable,price
0,34.103701,-118.332241,1,13,2.0,3,2,150,2,1,100,1,350
1,34.099484,-118.331645,1,8,2.0,2,4,150,1,11,96,1,190
2,34.104321,-118.329662,1,4,1.0,0,1,55,1,1,80,0,85
3,34.101028,-118.317848,0,2,1.0,1,1,20,1,8,98,0,75
4,34.098292,-118.324980,1,2,1.0,1,1,20,1,11,96,0,130


In [3]:
test.head()

,latitude,longitude,Entire home,accommodates,bathrooms,bedrooms,beds,cleaning_fee,minimum_nights,number_of_reviews,review_scores_rating,instant_bookable,price
0,34.100604,-118.341787,0,2,1.0,1,1,40,1,261,96,1,100
1,34.100607,-118.350583,1,8,2.0,2,2,100,2,10,98,0,300
2,34.100610,-118.347617,1,2,1.0,1,1,80,2,1,100,1,125
3,34.100611,-118.342180,1,3,1.0,0,2,55,2,54,97,1,169
4,34.100618,-118.342791,1,4,1.0,1,1,70,2,233,92,1,119


In [30]:
actual = test["price"]
test.drop('price', axis=1,inplace=True)
x = pd.DataFrame(train)
y = train["price"]
m = 12
n = len(y)

In [13]:
len(test)

699

# Model 1

In [31]:
mod = Model()
beta = mod.addVars(m)
z = mod.addVars(n, lb = -GRB.INFINITY)

for i in range(n):
    mod.addConstr(z[i] >= y[i] - sum(beta[j] * x.iloc[i,j] for j in range(m))) 
    mod.addConstr(z[i] >= sum(beta[j] * x.iloc[i,j] for j in range(m)) - y[i])

mod.setObjective(sum(z[i] / n for i in range(n)), GRB.MINIMIZE)
mod.update()
mod.optimize()

betas = []
for j in range(m):
    betas.append(beta[j].x)
    
test = np.asarray(test)

prediction_error = sum(abs(actual[i] - sum(betas[j] * test[i,j] for j in range(m))) for i in range(len(test))) / len(test)
print('Prediction Error:', prediction_error)

Gurobi Optimizer version 10.0.0 build v10.0.0rc2 (mac64[rosetta2])

CPU model: Apple M2
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 3400 rows, 1712 columns and 41372 nonzeros
Model fingerprint: 0x85d2b015
Coefficient statistics:
  Matrix range     [5e-01, 5e+02]
  Objective range  [6e-04, 6e-04]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+01, 2e+03]
Presolve removed 1700 rows and 0 columns
Presolve time: 0.01s
Presolved: 1700 rows, 1712 columns, 20686 nonzeros

Concurrent LP optimizer: primal simplex, dual simplex, and barrier
Showing barrier log only...

Ordering time: 0.00s

Barrier statistics:
 Dense cols : 12
 AA' NZ     : 1.433e+04
 Factor NZ  : 1.570e+04 (roughly 1 MB of memory)
 Factor Ops : 1.911e+05 (less than 1 second per iteration)
 Threads    : 1

                  Objective                Residual
Iter       Primal          Dual         Primal    Dual     Compl     Time
   0  -1.15719031e+03 -1.98951966e-1

# Model 2

In [34]:
mod = Model()
beta = mod.addVars(n)
z = mod.addVars(n, lb = -GRB.INFINITY)
coef = mod.addVars(m,vtype = GRB.BINARY)

mod.addConstr(sum(coef[j] for j in range(m)) <= 3)
for j in range(m):
    mod.addConstr(beta[j] <= coef[j] * 100000)
    mod.addConstr(-beta[j] <= coef[j] * 100000)
    
for i in range(n):
    mod.addConstr(z[i] >= sum(beta[j] * x.iloc[i,j] for j in range(m)) - y[i])
    mod.addConstr(z[i] >= y[i] - sum(beta[j] * x.iloc[i,j] for j in range(m)))

mod.setObjective(sum(z[i] / n for i in range(n)), GRB.MINIMIZE)
mod.update()
mod.optimize()

col = x.columns
for j in range(m):
    if coef[j].x == 1:
        print ('Coefficient for',col[j],':',beta[j].x)
        
betas = []        
for j in range(m):
    betas.append(beta[j].x)
    
test = np.asarray(test)

prediction_error = sum(abs(actual[i] - sum(betas[j] * test[i,j] for j in range (m))) for i in range(699)) / 699
print('Prediction Error:',prediction_error)

Gurobi Optimizer version 10.0.0 build v10.0.0rc2 (mac64[rosetta2])

CPU model: Apple M2
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 3425 rows, 3412 columns and 41432 nonzeros
Model fingerprint: 0xcf03fba7
Variable types: 3400 continuous, 12 integer (12 binary)
Coefficient statistics:
  Matrix range     [5e-01, 1e+05]
  Objective range  [6e-04, 6e-04]
  Bounds range     [1e+00, 1e+00]
  RHS range        [3e+00, 2e+03]
Found heuristic solution: objective 144.9682353
Presolve removed 840 rows and 2102 columns
Presolve time: 0.04s
Presolved: 2585 rows, 1310 columns, 31274 nonzeros
Variable types: 1298 continuous, 12 integer (12 binary)

Root relaxation: objective 3.642625e+01, 2329 iterations, 0.22 seconds (0.52 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0   36.42625    0    5  144.96824   36.42625  74.9% 

# Model 3

In [38]:
mod = Model()
beta = mod.addVars(n)
z = mod.addVars(n, lb = -GRB.INFINITY)
coef = mod.addVars(m,vtype = GRB.BINARY)

mod.addConstr(sum(coef[j] for j in range(m)) <= 3)
mod.addConstr(coef[6] == 1)

for j in range(m):
    mod.addConstr(beta[j] <= coef[j] * 100000)
    mod.addConstr(-beta[j] <= coef[j] * 100000)

for i in range(n):
    mod.addConstr(z[i] >= sum(beta[j] * x.iloc[i,j] for j in range(m)) - y[i])
    mod.addConstr(z[i] >= y[i] - sum(beta[j] * x.iloc[i,j] for j in range(m)))

mod.setObjective(sum(z[i] / n for i in range(n)), GRB.MINIMIZE)
mod.update()
mod.optimize()

col = x.columns
for j in range(m):
    if coef[j].x == 1:
        print ('Coefficient for', col[j],':',beta[j].x)
        
betas = []        
for j in range(m):
    betas.append(beta[j].x)

test = np.asarray(test)

prediction_error = sum(abs(actual[i] - sum(betas[j] * test[i,j] for j in range (m))) for i in range(699))/699
print('Prediction Error:',prediction_error)

Gurobi Optimizer version 10.0.0 build v10.0.0rc2 (mac64[rosetta2])

CPU model: Apple M2
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 3426 rows, 3412 columns and 41433 nonzeros
Model fingerprint: 0x3f60590c
Variable types: 3400 continuous, 12 integer (12 binary)
Coefficient statistics:
  Matrix range     [5e-01, 1e+05]
  Objective range  [6e-04, 6e-04]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+03]
Found heuristic solution: objective 144.9682353
Presolve removed 842 rows and 2103 columns
Presolve time: 0.04s
Presolved: 2584 rows, 1309 columns, 31271 nonzeros
Variable types: 1298 continuous, 11 integer (11 binary)

Root relaxation: objective 3.642625e+01, 2329 iterations, 0.22 seconds (0.52 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0   36.42625    0    6  144.96824   36.42625  74.9% 

While "Accomdates" is appeared in Model 2 as an important variable, it is no longer in model 3. The reason for this might be due to the fact that Accomodate was the least important variable among the three variables in model 2. Thus, if you set the number of beds as one of the three variables in the model, "Accomodates" variable is dropped. 